<a href="https://colab.research.google.com/github/kenkyl/Redis-Workshops-Kenkyl/blob/main/04-Large_Language_Model/04-Large_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large Language Models

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

In this notebook you'll be using two LLMs. OpenAI ChatGPT `gpt-3.5-turbo` and Self - hosted in - notebook `databricks/dolly-v2`. 

In [ ]:
%pip -q install cohere tiktoken openai accelerate transformers sentence-transformers


Initialize OpenAI. You need to supply the OpenAI API key (starts with `sk-...`) when prompted. You can find your API key at https://platform.openai.com/account/api-keys

In [ ]:
import openai
import os
import getpass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY","")
if OPENAI_API_KEY == "":
    key=getpass.getpass(prompt='OpenAI Key: ', stream=None)
    os.environ['OPENAI_API_KEY']=key
    
openai.api_key = os.getenv("OPENAI_API_KEY")

Initialize `databricks/dolly-v2-3b` via [HuggingFace](https://huggingface.co/databricks/dolly-v2-3b). Multiple progressively more powerful models are available, including 3b, 7b and 12b (referring to Billions of parameters). `dolly-v2-3b` is the only model in the family that would fit in the memory and GPU available in a free Google Colab instance.

Loading and initializing the model can take few minutes.

In [ ]:
import torch
from transformers import pipeline

dolly_completion = pipeline(model="databricks/dolly-v2-3b", 
                         torch_dtype=torch.bfloat16, 
                         trust_remote_code=True, 
                         device_map="auto")


Helper function for OpenAI ChatGPT model

In [4]:
def openai_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response

# Create the prompt

Prompt contains instructions, context and the question. Feel free to experiment with the prompt and see the difference in responses from different models.

News article used in this example: https://techcrunch.com/2023/10/31/toyota-to-invest-another-8b-into-north-carolina-ev-battery-factory/

In [ ]:
context = """
Toyota said Tuesday it will invest another $8 billion into its first EV battery factory in North America, as the Japanese automaker tries to ramp up its electrification program and introduce 30 battery electric models globally by the end of the decade.
The North Carolina-based factory, which is slated to go into production in 2025, is now valued at $13.9 billion, according to the company. 
That’s a jump from Toyota’s initial plans to make a $1.29 billion investment to build a North American factory that will make batteries for hybrid electric vehicles and battery electric vehicles.
This latest investment will add eight battery electric and plug-in hybrid battery production lines to the facility located on 1,825 acres in Liberty, North Carolina. 
Once completed, the factory will have 10 lines and will reach a total production capacity of 30 GWh annually by 2030. To put that into perspective, the so-called Tesla gigafactory, which is a joint venture between the automaker and Panasonic, has the capacity to make 35 GWh of cells annually.
"""

question="What is Toyota building in North Carolina?"

prompt = f"""
Instruction: Use only information in the following context to answer the question at the end. 
If you don't know, say that you do not know. 
 
Context:  {context}
 
Question: {question}
 
Response:
"""
print(prompt)

res = dolly_completion(prompt)
print("Dolly:")
print(res[0]['generated_text'])


res = openai_completion(prompt)
print("\nOpenAI:")
print(res.choices[0].message["content"])

## TODO:

Some ideas for you to try: 

- Provide more information into the context until you hit the token limit of the model.

- Replace the entire prompt with a simple task like "Tell me about Austin, Texas"

- provoke hallucinations "What was the name of the first elephant to walk on the moon?"

- add "Respond in French/Spanish" to the prompt.

In [ ]:
prompt = "Tell me about Austin, Texas"

res = dolly_completion(prompt)
print("Dolly:")
print(res[0]['generated_text'])


res = openai_completion(prompt)
print("\nOpenAI:")
print(res.choices[0].message["content"])